In [13]:
# Importing dependencies
import os
import glob
import pandas as pd

In [14]:
# Set working dir
os.chdir("/Users/williamforsyth/Documents/uc_davis/Homework_Repos/tableau-citi-bike-analysis/data/raw_data")

In [15]:
# Find all csf files in working dir, using global pattern matching
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [16]:
# combining data from csv files into list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [17]:
# exporting list of all csv data to a csv file
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [18]:
# a quick check of the combined csv
combined_csv.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336.0,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389.0,Broadway & Berry St,40.710446,-73.965251,3562.0,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255.0,8 Ave & W 31 St,40.750585,-73.994685,505.0,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367.0,E 53 St & Lexington Ave,40.758281,-73.970694,497.0,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2


In [19]:
# Splitting Gender (0, male is 1, and female is 2) & Usertype based on Citi Bike's documentation and in preparation for aggregation.
combined_csv["gender_unknown"] = combined_csv["gender"] == 0
combined_csv["gender_unknown"] = combined_csv["gender_unknown"].astype(int)

combined_csv["gender_male"] = combined_csv["gender"] == 1
combined_csv["gender_male"] = combined_csv["gender_male"].astype(int)

combined_csv["gender_female"] = combined_csv["gender"] == 2
combined_csv["gender_female"] = combined_csv["gender_female"].astype(int)

combined_csv["subscriber"] = combined_csv["usertype"] == "Subscriber"
combined_csv["subscriber"] = combined_csv["subscriber"].astype(int)

combined_csv["customer"] = combined_csv["usertype"] == "Customer"
combined_csv["customer"] = combined_csv["customer"].astype(int)

combined_csv.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,gender_unknown,gender_male,gender_female,subscriber,customer
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,3419.0,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,0,0,1,0,1
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336.0,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,1,0,0,1,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389.0,Broadway & Berry St,40.710446,-73.965251,3562.0,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,0,0,1,0,1
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255.0,8 Ave & W 31 St,40.750585,-73.994685,505.0,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,1,0,0,0,1
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367.0,E 53 St & Lexington Ave,40.758281,-73.970694,497.0,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,0,0,1,0,1
